In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(survminer)
library(rlang)
library(cowplot)
library(patchwork)
library(ggpubr)

# Read in base data

In [ ]:
ready <- fread(paste0(SHARE_DIR, "fisher_base.csv")) 

In [ ]:
highlights <- fread(paste0(SHARE_DIR,"share_with_fran_update.csv"))

In [ ]:
go <- highlights %>% se(cohortGo, feature)

### Survival Images

In [ ]:
figurer <- function(x, cohort ) {
    data <- ready %>% fi(cohortGo == !!cohort) #%>% drop_na( !!sym(x) )
    surv_formula <- expr(Surv(daysToPfsEvent, pfsEvent) ~ !!sym(x))
    pval <- signif(survdiff(eval(surv_formula), data = data)$pvalue,2)
    fits <- survfit(eval(surv_formula), data = data)

    #fit
    oo <- 
    ggsurvplot(
     fits,
     data = data,
     palette = "Dark2",
     conf.int = TRUE, 
     risk.table = TRUE, 
     pval.coord = c(700, .95), 
     break.time.by = 300, 
     ggtheme = theme_minimal(),
     xlab = "Days", 
     ylab = "Survival Probability", 
     title = cohort) 

    oo$plot <- oo$plot + annotate("text", x = 500, y = 0.9, label = paste0("Log-rank    \np-value = ", pval), size = 5) + theme(plot.title = element_text(hjust = 0.5)) + guides(color = guide_legend(nrow = 2, byrow = TRUE))
    as_ggplot(gridExtra::arrangeGrob(oo$plot, oo$table, layout_matrix = matrix(c(1,1,1,1,1,1,1,2,2)))) + theme(plot.margin = margin(t = 20, r = 20, b = 20, l = 20))
}

### Problems with the Interaction Features

In [ ]:
figures <- list()
for( i in seq(16)){
    cohortGo <- as.character(go[i,1])
    feature <- as.character(go[i,2])
    figures[[as.character(i)]] <- figurer( x = feature, cohort = cohortGo)
}
see <- wrap_plots(figures, ncol = 4)

In [ ]:
options(repr.plot.height = 26, repr.plot.width = 26) 
see

In [ ]:
ggsave(paste0(FIG_DIR, "survival_highlight.png"), height = 26, width = 26)

In [ ]:
paste0(FIG_DIR, "survival_highlight.png")